# Générons une prédiction valide pour le challenge

Imports nécessaires au notebook

In [1]:
import time
import re
import pandas as pd
import numpy as np
import nltk
import sklearn
import pickle
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
import tensorflow_datasets as tfds
from multiprocessing import Manager, Process

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

Nom de l'équipe : 

In [2]:
TEAM_NAME = "Wasabi"
assert re.match("^\w+$", TEAM_NAME) is not None, "Nom d'équipe invalide"

In [3]:
# Chargement des données d'entrainement
train = pd.read_csv("train.csv.gz")

In [4]:
# Chargement des données à prédire
test = pd.read_csv("soutenance.csv.gz")

Combien y a t'il de catégories produits distinctes dans le dataset?

In [5]:
len(train.category.value_counts())

600

## La fonction de publication de vos prédictions

In [6]:
# predictions est un dataframe qui contient au moins les colonnes "idp" et "category"
# Ca écrit simplement le fichier dans le répertoire d'évaluation
def publish_results(predictions):
    now = int(time.time())
    assert re.match("^\w+$", TEAM_NAME) is not None
    filename = f"/home/cisd-jacq/projet/evaluation/prediction-{TEAM_NAME}-{now}.csv.gz"
    predictions[["idp", "category"]].to_csv(filename, index=False, compression="gzip")
    return filename.split("/")[-1]

## La fonction d'évaluation de l'erreur

Vous pouvez utiliser cette fonction pour estimer sur un sous ensemble du dataset train quelle est la précision de votre modèle.

In [7]:
# Plus ce score est grand moins on est content
def error(real_category, predicted_category):
    # On a trouvé la bonne catégorie
    if real_category == predicted_category:
        return 0
    
    # On extrait les sous catégories
    real_categories = real_category.split(" > ")
    pred_categories = predicted_category.split(" > ")
    # On flag les catégories adultes
    adult_categories = ['ADULTE - EROTIQUE', 'VIN - ALCOOL - LIQUIDES']
    real_is_adult = real_categories[0] in adult_categories
    pred_is_adult = pred_categories[0] in adult_categories
    
    # Attention non symmétrie de l'erreur !
    if real_is_adult and not pred_is_adult:
        error = 10_000
        return error
    
    # On identifie à quel niveau on s'est trompé
    for real, pred, error in zip(real_categories, pred_categories, [100, 10, 1]):
        if real != pred:
            return error
    raise ValueError("Catégories différentes, mais aucune différence trouvée")

# Une prédiction moins dummy


## 1. Travaillons sur 10% du dataset de train

Commencons par itérer rapidement sur le jeu de données. Une fois qu'on aurra un modèle qui nous convient on pourra travailler sur plus de volumétrie.

Générez train_subset, un sample de train faisant 10% de sa taille

In [8]:
train_subset = train.sample(frac=0.1)

## 2. Limitons les classes

600 classes c'est beaucoup trop. 

On a vu que les catégories contiennent une hiérarchie "catégorie 1 > catégorie 2 > catégorie 3", commencons par travailler uniquement sur les catégories 1.

Générez la colonne "category_1" dans train_subset à partir de la colonne "category" qui contient uniquement la catégorie 1 du produit

In [9]:
train_subset["category_1"] = train_subset["category"].str.split(">", n=1, expand=True)[0]

In [10]:
categories_freq = {}
for el in train_subset.iterrows():
    cat1 = el[1]["category_1"]
    if cat1 not in categories_freq:
        categories_freq[cat1] = {"max_freq" : 0, "max_cat" : el[1]["category"]}
    
    if el[1]["category"] not in categories_freq[cat1]:
        categories_freq[cat1][el[1]["category"]] = 1
    else :
        categories_freq[cat1][el[1]["category"]] += 1
        if categories_freq[cat1]["max_freq"] < categories_freq[cat1][el[1]["category"]]:
            categories_freq[cat1]["max_freq"] += 1 
            categories_freq[cat1]["max_cat"] = el[1]["category"]


Combien de "category_1" distinctes vous avez?

In [11]:
nbcat1 = len(np.unique(train_subset["category_1"]))

C'est bien plus raisonnable pour commencer.

## 3. Oh oh il y a pleins de features.

Il y a pleins de features différentes.

- 2 champs de texte title et description 
- 1 champ float : price.

On va se contenter de travailler uniquement avec "title" pour commencer.

Générez les features : X_train

In [12]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))

def tokenize_text(line):
    tokens = nltk.word_tokenize(line)
    new_tokens = []
    pattern = '\w'
    for token in tokens:
        if len(token) > 1 and token not in stop_words and re.match(pattern, token):
            new_tokens.append(token.lower())
    return new_tokens


[nltk_data] Downloading package stopwords to /home/cisd-
[nltk_data]     cazalet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
if 'title_tokens' in train_subset.columns:
    train_subset = train_subset.drop(columns=["title_tokens"])
    
train_subset.insert(train_subset.shape[1], "title_tokens", np.array([tokenize_text(x) for x in train_subset["title"].to_numpy()]))

In [14]:
def assign_id_to_words(data, field):
    words_to_id = {}
    nextid = 0
    for row in data[field]:
        for word in row:
            if word not in words_to_id:
                words_to_id[word] = nextid
                nextid += 1
    return words_to_id

On tokenise la colonne 'title' de l'ensemble test

In [15]:
if 'title_tokens' in test:
    test = test.drop(columns=["title_tokens"])

nproc = 20
manager = Manager()
title_tokens = manager.list()
titles = test["title"].to_numpy()

jobs = []
def vprocess_text(start, end):
    global title_tokens
    global titles
    for i in range(start, end):
        title_tokens.append(tokenize_text(titles[i]))

for pid in range(nproc):
    job = Process(target=vprocess_text, args=[int(pid*len(titles)/nproc), len(titles) if pid+1==nproc else int((pid+1)*len(titles)/nproc)])
    job.start()
    jobs.append(job)

for job in jobs:
    job.join()

del titles
test.insert(test.shape[1], "title_tokens", np.array(title_tokens))

In [16]:
# if 'title_tokens' in test:
#     test = test.drop(columns=["title_tokens"])
# test.insert(test.shape[1], "title_tokens", np.array([tokenize_text(x) for x in test["title"].to_numpy()]))

In [17]:
words_to_id = assign_id_to_words(train_subset, "title_tokens")

In [18]:
categories_to_id = {}
nextid = 0
for cat in train_subset["category_1"]:
    if cat not in categories_to_id:
         categories_to_id[cat] = nextid
         nextid += 1

In [19]:
id_to_category = {v: k for k, v in categories_to_id.items()}

In [20]:
def bag_of_words_to_vector(bag_of_words, words_to_id):
    features = np.zeros(len(words_to_id))
    for word in bag_of_words:
        if word in words_to_id:
            features[words_to_id[word]] = 1
    return features

À partir de l'ensemble d'entrainement, on créer le X_train et le Y_train pour entrainer notre modèle

In [ ]:
manager = Manager()

X_train = manager.list()
y_train = manager.list()

def build_X_train():
    global X_train
    for row in train_subset.iterrows():
        X_train.append(bag_of_words_to_vector(row[1]["title_tokens"], words_to_id))
    X_train = np.array(X_train)
    
def build_y_train():
    global y_train
    for row in train_subset.iterrows():
        y_train.append(categories_to_id[row[1]["category_1"]])
    y_train = np.array(y_train)

p_X_train = Process(target=build_X_train)
p_y_train = Process(target=build_y_train)
p_X_train.start()
p_y_train.start()
p_X_train.join()
p_y_train.join()

X_train = np.array(X_train)
y_train = np.array(y_train)

features_dim = len(X_train[0])

## 4. Calculez un modèle avec ces simplifications

On entraine une régression Logistique

In [ ]:
model = sklearn.linear_model.LogisticRegression()

model.fit(X_train, y_train)

## 5. Sauvegardez votre modèle

Pour ne pas devoir recommencer à chaque fois tout ce dur labeur, et cette longue attente, vous pouvez sauvegarder votre modèle et votre vectorizer.

La prochaine fois vous n'aurez qu'à les recharger pour faire directement vos prédictions (c'est ce qui est attendu pour la soutenance, sinon le timing sera trop serré) 

La documentation : https://scikit-learn.org/stable/modules/model_persistence.html

In [ ]:
filename = "logistic.model"
pickle.dump(model, open(filename, 'wb'))

In [ ]:
model = pickle.load(open(filename, 'rb'))

## 6.. Calculez votre prédiction

Appliquez votre modèle sur les données test

In [ ]:
# confusion_matrix = sklearn.metrics.confusion_matrix(y_train, model.predict(X_train))
#confusion_matrix
#del confusion_matrix

In [ ]:
# sklearn.model_selection.cross_validate(model, X_train, y_train, cv=5)

Si on a pas assez d'espace disponible pour la prédiction : 

In [ ]:
#del X_train
#del y_train

## 7. Soumettez votre prédiction.

*Pas si vite* : Vous ne prédisez que la catégorie 1. Le script d'évaluation attend une catégorie complète...

C'est simple, pour chaque catégorie 1, choisissez la catégorie de votre choix qui commence par cette "categorie 1".

Modifiez votre prédiction, y_pred, en conséquence.

Soumettez votre prédiction :

In [ ]:
test

In [ ]:
y_pred = []
test_titles = test["title_tokens"].to_numpy()

batch_size = 80000
i = 0
while i < test.shape[0]:
    batch = np.array([bag_of_words_to_vector(title_tokens, words_to_id) for title_tokens in test_titles[i:min(i+batch_size, test.shape[0])]])
    batch_pred = model.predict(batch)
    y_pred = y_pred + list(map(lambda cat_id: categories_freq[id_to_category[cat_id]]["max_cat"], batch_pred))
    i += batch_size
y_pred = np.array(y_pred)

In [ ]:
prediction = pd.DataFrame({"idp": test["idp"], "category" : y_pred})

In [ ]:
publish_results(prediction)

Vous pouvez regardez vos scores en exécutant le notebook [Leaderboard.ipynb](Leaderboard.ipynb). Les données sont mises à jour toutes les 30min.

# Maintenant à vous de jouer

Vous pouvez commencer par travailler sur plus de volumétrie que 5% du dataset. Mais maintenant le challenge commence.

Si vous ne savez pas par où commencer suivez le déroulement des 2 premiers TPs, en prenant garde à la volumétrie. Ils sont disponibles dans le répertoire ~cisd-jacq/TP/

Contrairement aux TPs vous n'avez pas d'information sur les données de test. (Mis à part le score calculé toute les 30min).

Pour évaluer votre modèle et l'améliorer vous pouvez utiliser les données de train pour crééer un ensemble d'entrainement et un ensemble de validation. 

Vous pourrez alors évaluer plus rapidement vos modèles et identifier quelles sont les catégories sur lesquelles vous devez vous améliorer.

Il n'y a pas que la Régression Logistique dans la vie, essayez d'autres modèles. Je vous ai fait travailler avec sklearn, mais il existe aussi d'autres librairies.

Pour paralléliser vos calculs :
- multiprocessing : https://docs.python.org/3/library/multiprocessing.html
- dask : https://dask.org/ + https://distributed.dask.org/en/latest/ 

## Avec un tokenizer différent

In [21]:
# Générateur de corpus
def corpus_generator():
    for row in train.iterrows():
        for word in row[1]["title"]:
            yield word

In [22]:
vocab_filename = "train.vocab"

Créer un encodeur de texte à partir d'un corpus de texte composé des titres

In [23]:
#encoder = tfds.features.text.SubwordTextEncoder.build_from_corpus(corpus_generator(), target_vocab_size=2**15)
#encoder.save_to_file(vocab_filename)

In [24]:
encoder = tfds.features.text.SubwordTextEncoder.load_from_file(vocab_filename)

In [25]:
X_train = []
dim = 0

for row in train_subset.iterrows():
    encoded = encoder.encode(row[1]["title"])
    dim = max(dim, len(encoded))
    X_train.append(encoded)

X_train = [x+[0.0 for i in range(dim-len(x)) ] for x in X_train]
X_train = np.array(X_train)

X_train.shape

(30000, 141)

In [28]:
try:
    y_train
except NameError:
    y_train = []
    for row in train_subset.iterrows():
        y_train.append(categories_to_id[row[1]["category_1"]])
    y_train = np.array(y_train)

    y_train = np.array(y_train)
else:
    print("well, it WASN'T defined after all!")



In [ ]:
model = sklearn.linear_model.LogisticRegression()
model.fit(X_train, y_train)

On sauvegarde le modèle

In [ ]:
filename = "logistic_with_other_tokenizer.model"
pickle.dump(model, open(filename, 'wb'))

In [ ]:
model = pickle.load(open(filename, 'rb'))

In [ ]:
y_pred = []
test_titles = test["title"].to_numpy()

batch_size = 80000
i = 0
while i < test.shape[0]:
    batch = []
    for title in test_titles[i:min(i+batch_size, test.shape[0])]:
        vec = encoder.encode(title)
        if len(vec) < dim:
            vec = vec + [0.0 for i in range(dim-len(vec))]
        elif len(vec) > dim:
            vec = vec[0:dim]
        if len(vec) != 146:
            print(len(vec))
        batch.append(vec)
    batch = np.array(batch)
    print(batch.shape)
    batch_pred = model.predict(batch)
    y_pred = y_pred + list(map(lambda cat_id: categories_freq[id_to_category[cat_id]]["max_cat"], batch_pred))
    i += batch_size
y_pred = np.array(y_pred)

In [ ]:
X_test[0].shape

In [ ]:
X_test.shape

In [ ]:
prediction = pd.DataFrame({"idp": test["idp"], "category" : y_pred})
publish_results(prediction)

## Réseau de neuronne récurrent

On a réalisé l'implémentation d'un réseau de neuronne récurrent comme expliqué dans cet article : https://towardsdatascience.com/multi-class-text-classification-with-lstm-using-tensorflow-2-0-d88627c10a35


In [29]:
def build_model(embedding_dim=(len(X_train[0])+1)):
    model = tensorflow.keras.Sequential([
        tensorflow.keras.layers.Embedding(encoder.vocab_size, embedding_dim),
        tensorflow.keras.layers.Bidirectional(tensorflow.keras.layers.LSTM(embedding_dim)),
        tensorflow.keras.layers.Dense(embedding_dim, activation='relu'),
        tensorflow.keras.layers.Dense(len(np.unique(train_subset["category_1"])), activation='softmax')
    ])
    return model

model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 142)         63048     
_________________________________________________________________
bidirectional (Bidirectional (None, 284)               323760    
_________________________________________________________________
dense (Dense)                (None, 142)               40470     
_________________________________________________________________
dense_1 (Dense)              (None, 30)                4290      
Total params: 431,568
Trainable params: 431,568
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
num_epochs = 17
history = model.fit(X_train, y_train, epochs=num_epochs, verbose=2)

Train on 30000 samples
Epoch 1/17
30000/30000 - 129s - loss: 2.2128 - accuracy: 0.3623
Epoch 2/17
30000/30000 - 129s - loss: 1.8964 - accuracy: 0.4512
Epoch 3/17
30000/30000 - 129s - loss: 1.6285 - accuracy: 0.5291
Epoch 4/17
30000/30000 - 129s - loss: 1.4103 - accuracy: 0.5951
Epoch 5/17
30000/30000 - 129s - loss: 1.2119 - accuracy: 0.6523
Epoch 6/17
30000/30000 - 129s - loss: 1.0588 - accuracy: 0.6943
Epoch 7/17
30000/30000 - 128s - loss: 0.9345 - accuracy: 0.7289
Epoch 8/17
30000/30000 - 128s - loss: 0.8116 - accuracy: 0.7637
Epoch 9/17
30000/30000 - 127s - loss: 0.7207 - accuracy: 0.7894
Epoch 10/17
30000/30000 - 126s - loss: 0.6293 - accuracy: 0.8148
Epoch 11/17
30000/30000 - 126s - loss: 0.5411 - accuracy: 0.8396
Epoch 12/17
30000/30000 - 126s - loss: 0.4746 - accuracy: 0.8569
Epoch 13/17
30000/30000 - 126s - loss: 0.4101 - accuracy: 0.8787
Epoch 14/17
30000/30000 - 125s - loss: 0.3699 - accuracy: 0.8879
Epoch 15/17
30000/30000 - 126s - loss: 0.2994 - accuracy: 0.9078
Epoch 16/17

On sauvegarde le modèle

In [40]:
filename = "rnn.model"
model.save('rnn.h5') 

In [42]:
new_model = tensorflow.keras.models.load_model('rnn.h5')

In [43]:
vec = encoder.encode(test["title"].to_numpy()[0])
prediction = model.predict([vec])
print(prediction.shape)
print(prediction)
print(np.argmax(prediction))

(1, 30)
[[7.0908445e-18 6.6020569e-08 3.6599912e-09 5.2164545e-08 6.3250676e-09
  2.0278440e-11 3.6214634e-10 4.8039500e-10 2.1252086e-13 1.0973860e-07
  1.0908095e-05 5.1187732e-07 1.3766445e-05 8.3147403e-05 1.2835205e-10
  1.9285442e-16 3.1951049e-04 5.8961655e-06 2.4381713e-10 4.7433235e-09
  8.7174544e-07 1.5900586e-11 8.8108670e-12 5.6281412e-04 9.9885714e-01
  6.1119718e-09 1.4506807e-04 1.3922382e-11 1.0400946e-07 3.8983330e-11]]
24


In [38]:
y_pred = []
test_titles = test["title"].to_numpy()

batch_size = 80000
i = 0
while i < test.shape[0]:
    batch = []
    for title in test_titles[i:min(i+batch_size, test.shape[0])]:
        vec = encoder.encode(title)
        if len(vec) < dim:
            vec = vec + [0.0 for i in range(dim-len(vec))]
        elif len(vec) > dim:
            vec = vec[0:dim]
        batch.append(vec)
    batch = np.array(batch)
    batch_pred = model.predict(batch)
    y_pred = y_pred + list(map(lambda pred: categories_freq[id_to_category[np.argmax(pred
                                                                                    )]]["max_cat"], batch_pred))
    i += batch_size
y_pred = np.array(y_pred)

In [44]:
prediction = pd.DataFrame({"idp": test["idp"], "category" : y_pred})
publish_results(prediction)

In [48]:
error(test["category"],prediction["category"])

,idp,title,description,price,title_tokens
0,510714,Beige et marron Portefeuille femme Iqzco,"Matériau extérieur: matière synthétique, intér...",45.99,"[beige, marron, portefeuille, femme, iqzco]"
1,2235312,"Hama Étui Eva Pour Disque Dur Externe 2,5"" ...","HAMA ÉTUI EVA POUR DISQUE DUR EXTERNE 2,5"" ...",14.57,"[hama, étui, eva, pour, disque, dur, externe, ..."
2,5235844,"Bandeau Cheveux Femme Vintage, Bandeau Imprimé...",Caractéristiques:100% neuf et haute qualitéLe ...,8.40,"[bandeau, cheveux, femme, vintage, bandeau, im..."
3,6336566,3pcs plumes de duvet de polyester vers le bas ...,"Tissu polyester: doux et confortable, respiran...",24.93,"[3pcs, plumes, duvet, polyester, vers, bas, co..."
4,5214878,Débardeur rouge décoré de fleurs,Débardeur côtelé et moulant.Modèle avec bretel...,14.99,"[débardeur, rouge, décoré, fleurs]"
...,...,...,...,...,...
99995,1049028,Autocollant de voiture volant Hawk auto Truck ...,Car Decal volant faucon Auto Camion capot côté...,0.93,"[applique, mural, luminaire, contemporain, 6w,..."
99996,3236083,Canne A Peche 138H 2 0 Prov Bend Baitholder Sn...,Canne A Peche 138H 2/0 Prov Bend Baitholder Sn...,42.99,"[harceleurs, l'école, bureau]"
99997,837827,Aléa,Jan Kjrstad Du monde entier,23.20,"[jarretière, optique, duplex, 2.0, mm, multi, ..."
99998,2150106,Pour Sony Xperia Z5 : Oreillette Bluetooth Ori...,Oreillette sans fil bluetooth v4.1 ultra légèr...,26.99,"[maison, poupée, bois, diy, modèle, miniature,..."


# Ce qu'il reste à faire
- utiliser la base de donnée entière
- utiliser tous les champs
- paralléliser encore plus, probablement avec GPU
- tester des variations du modèle pour prendre le meilleur